# **Gradient Clipping**

## **Libraries**

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import numpy as np
import torch
import time

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## **Settings**

In [3]:
# Device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.01
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 256
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32
num_classes = 10

## **MNIST dataset**

In [4]:
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())

In [5]:
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

In [6]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [7]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100

## **Model**

In [8]:
class MultilayerPerceptron(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_3 = torch.nn.Linear(num_hidden_2, num_hidden_3)
        self.linear_4 = torch.nn.Linear(num_hidden_3, num_hidden_4)
        self.linear_out = torch.nn.Linear(num_hidden_4, num_classes)
        
    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        out = self.linear_3(out)
        out = F.relu(out)
        out = self.linear_4(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas

## **1 - Basic Clipping**

In [9]:
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features, num_classes=num_classes)
model = model.to(device)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [11]:
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 1.)
        
        optimizer.step()
        
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 2.3278
Epoch: 001/010 | Batch 050/938 | Cost: 0.7310
Epoch: 001/010 | Batch 100/938 | Cost: 0.4491
Epoch: 001/010 | Batch 150/938 | Cost: 0.1983
Epoch: 001/010 | Batch 200/938 | Cost: 0.4208
Epoch: 001/010 | Batch 250/938 | Cost: 0.1627
Epoch: 001/010 | Batch 300/938 | Cost: 0.3228
Epoch: 001/010 | Batch 350/938 | Cost: 0.1020
Epoch: 001/010 | Batch 400/938 | Cost: 0.2994
Epoch: 001/010 | Batch 450/938 | Cost: 0.2947
Epoch: 001/010 | Batch 500/938 | Cost: 0.1846
Epoch: 001/010 | Batch 550/938 | Cost: 0.2925
Epoch: 001/010 | Batch 600/938 | Cost: 0.2301
Epoch: 001/010 | Batch 650/938 | Cost: 0.3788
Epoch: 001/010 | Batch 700/938 | Cost: 0.2000
Epoch: 001/010 | Batch 750/938 | Cost: 0.1393
Epoch: 001/010 | Batch 800/938 | Cost: 0.0938
Epoch: 001/010 | Batch 850/938 | Cost: 0.2002
Epoch: 001/010 | Batch 900/938 | Cost: 0.0697
Epoch: 001/010 training accuracy: 96.09%
Time elapsed: 0.61 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.2249
Epoch: 002/010 |

In [12]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.66%


## **2 - Custom Lower and Upper Bounds**

In [13]:
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features, num_classes=num_classes)

In [14]:
for p in model.parameters():
    p.register_hook(lambda grad: torch.clamp(grad, -0.1, 1.0))
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        optimizer.step()
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 2.3278
Epoch: 001/010 | Batch 050/938 | Cost: 0.7810
Epoch: 001/010 | Batch 100/938 | Cost: 0.4391
Epoch: 001/010 | Batch 150/938 | Cost: 0.1320
Epoch: 001/010 | Batch 200/938 | Cost: 0.3086
Epoch: 001/010 | Batch 250/938 | Cost: 0.1856
Epoch: 001/010 | Batch 300/938 | Cost: 0.2164
Epoch: 001/010 | Batch 350/938 | Cost: 0.1242
Epoch: 001/010 | Batch 400/938 | Cost: 0.2562
Epoch: 001/010 | Batch 450/938 | Cost: 0.2240
Epoch: 001/010 | Batch 500/938 | Cost: 0.3022
Epoch: 001/010 | Batch 550/938 | Cost: 0.2677
Epoch: 001/010 | Batch 600/938 | Cost: 0.3764
Epoch: 001/010 | Batch 650/938 | Cost: 0.3955
Epoch: 001/010 | Batch 700/938 | Cost: 0.1619
Epoch: 001/010 | Batch 750/938 | Cost: 0.2964
Epoch: 001/010 | Batch 800/938 | Cost: 0.1062
Epoch: 001/010 | Batch 850/938 | Cost: 0.3169
Epoch: 001/010 | Batch 900/938 | Cost: 0.1142
Epoch: 001/010 training accuracy: 96.49%
Time elapsed: 0.65 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.2291
Epoch: 002/010 |

In [16]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 94.88%


## **3 - Norm-clipping**

In [17]:
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features, num_classes=num_classes)
model = model.to(device)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [19]:
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1., norm_type=2)
        optimizer.step()
        
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 2.3278
Epoch: 001/010 | Batch 050/938 | Cost: 0.7573
Epoch: 001/010 | Batch 100/938 | Cost: 0.3458
Epoch: 001/010 | Batch 150/938 | Cost: 0.1678
Epoch: 001/010 | Batch 200/938 | Cost: 0.4749
Epoch: 001/010 | Batch 250/938 | Cost: 0.2108
Epoch: 001/010 | Batch 300/938 | Cost: 0.2096
Epoch: 001/010 | Batch 350/938 | Cost: 0.0609
Epoch: 001/010 | Batch 400/938 | Cost: 0.2998
Epoch: 001/010 | Batch 450/938 | Cost: 0.1873
Epoch: 001/010 | Batch 500/938 | Cost: 0.3246
Epoch: 001/010 | Batch 550/938 | Cost: 0.1418
Epoch: 001/010 | Batch 600/938 | Cost: 0.1558
Epoch: 001/010 | Batch 650/938 | Cost: 0.4166
Epoch: 001/010 | Batch 700/938 | Cost: 0.1930
Epoch: 001/010 | Batch 750/938 | Cost: 0.2087
Epoch: 001/010 | Batch 800/938 | Cost: 0.0546
Epoch: 001/010 | Batch 850/938 | Cost: 0.1882
Epoch: 001/010 | Batch 900/938 | Cost: 0.1557
Epoch: 001/010 training accuracy: 94.83%
Time elapsed: 0.72 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.4699
Epoch: 002/010 |

In [20]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 96.72%
